ใน ep ที่แล้ว

# 0. Install

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Install Library ที่จำเป็น

In [3]:
## Colab
! curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


เช็ค GPU

In [4]:
! nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



# 1. Import

Import Library ที่จะใช้ ในที่นี้คือ fastai และ fastai.text 

In [0]:
from fastai import *
from fastai.text import *
from fastai.callbacks import *
from fastai.callbacks.mem import *

In [0]:
# import fastai.utils.collect_env
# fastai.utils.collect_env.show_install()

สำหรับ [Google Colab](https://www.bualabs.com/archives/1687/what-is-colab-open-jupyter-notebook-in-github-on-google-colab-create-open-in-colab-button-colab-ep-1/) เราจะกำหนด path ที่เก็บ DataSet และ Mount Google Drive 

In [7]:
dataset = 'HUMAN_NUMBERS'

# Google Colab
config_path = Path('/content/drive')
data_path_base = Path('/content/datasets/')

data_path = data_path_base/dataset

from google.colab import drive
drive.mount(str(config_path))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# 2. Dataset

ในเคสนี้เราจะใช้ Human Numbers คือ รายการตัวเลข เขียนเป็นภาษาอังกฤษ 

In [8]:
path = untar_data(URLs.HUMAN_NUMBERS)
path.ls()

[PosixPath('/root/.fastai/data/human_numbers/train.txt'),
 PosixPath('/root/.fastai/data/human_numbers/valid.txt')]

In [12]:
! head {path}/train.txt

one 
two 
three 
four 
five 
six 
seven 
eight 
nine 
ten 


In [13]:
! tail {path}/train.txt

seven thousand nine hundred ninety 
seven thousand nine hundred ninety one 
seven thousand nine hundred ninety two 
seven thousand nine hundred ninety three 
seven thousand nine hundred ninety four 
seven thousand nine hundred ninety five 
seven thousand nine hundred ninety six 
seven thousand nine hundred ninety seven 
seven thousand nine hundred ninety eight 
seven thousand nine hundred ninety nine 


# 3. Preprocessing

## 3.1 อ่านไฟล์

ประกาศฟังก์ชัน สำหรับอ่านไฟล์มารวมคั่นด้วย Comma ใส่ List ไว้

In [0]:
def readnums(d): return [', '.join(o.strip() for o in open(path/d).readlines())]

In [16]:
train_txt = readnums('train.txt')
train_txt[0][:80]

'one, two, three, four, five, six, seven, eight, nine, ten, eleven, twelve, thirt'

In [20]:
valid_txt = readnums('valid.txt')
valid_txt[0][:80]

'eight thousand one, eight thousand two, eight thousand three, eight thousand fou'

## 3.2 Data Pipeline

สร้าง Training Set TextList (ItemList) และ Validation Set

In [27]:
train = TextList(train_txt, path=path)
valid = TextList(valid_txt, path=path)

train, valid

(TextList (1 items)
 eight thousand one, eight thousand two, eight thousand three, eight thousand four, eight thousand five, eight thousand six, eight thousand seven, eight thousand eight, eight thousand nine, eight thousand ten, eight thousand eleven, eight thousand twelve, eight thousand thirteen, eight thousand fourteen, eight thousand fifteen, eight thousand sixteen, eight thousand seventeen, eight thousand eighteen, eight thousand nineteen, eight thousand twenty, eight thousand twenty one, eight thousand twenty two, eight thousand twenty three, eight thousand twenty four, eight thousand twenty five, eight thousand twenty six, eight thousand twenty seven, eight thousand twenty eight, eight thousand twenty nine, eight thousand thirty, eight thousand thirty one, eight thousand thirty two, eight thousand thirty three, eight thousand thirty four, eight thousand thirty five, eight thousand thirty six, eight thousand thirty seven, eight thousand thirty eight, eight thousand thirty nine, 

รวม 2 Splitted ItemList ที่เราสร้างไว้

In [28]:
splitted = ItemLists(path=path, train=train, valid=valid)
splitted

ItemLists;

Train: TextList (1 items)
eight thousand one, eight thousand two, eight thousand three, eight thousand four, eight thousand five, eight thousand six, eight thousand seven, eight thousand eight, eight thousand nine, eight thousand ten, eight thousand eleven, eight thousand twelve, eight thousand thirteen, eight thousand fourteen, eight thousand fifteen, eight thousand sixteen, eight thousand seventeen, eight thousand eighteen, eight thousand nineteen, eight thousand twenty, eight thousand twenty one, eight thousand twenty two, eight thousand twenty three, eight thousand twenty four, eight thousand twenty five, eight thousand twenty six, eight thousand twenty seven, eight thousand twenty eight, eight thousand twenty nine, eight thousand thirty, eight thousand thirty one, eight thousand thirty two, eight thousand thirty three, eight thousand thirty four, eight thousand thirty five, eight thousand thirty six, eight thousand thirty seven, eight thousand thirty eight, eight thous

ใส่ Label สำหรับ Language Model ให้ทั้ง 2 ItemList

In [29]:
labeled_splitted = splitted.label_for_lm()
labeled_splitted

LabelLists;

Train: LabelList (1 items)
x: LMTextList
xxbos eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight thousand eighteen , eight thousand nineteen , eight thousand twenty , eight thousand twenty one , eight thousand twenty two , eight thousand twenty three , eight thousand twenty four , eight thousand twenty five , eight thousand twenty six , eight thousand twenty seven , eight thousand twenty eight , eight thousand twenty nine , eight thousand thirty , eight thousand thirty one , eight thousand thirty two , eight thousand thirty three , eight thousand thirty four , eight thousand thirty five , eight thousand thirty six , eight thous

สร้าง DataBunch

In [30]:
data = labeled_splitted.databunch()
data

TextLMDataBunch;

Train: LabelList (1 items)
x: LMTextList
xxbos eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight thousand eighteen , eight thousand nineteen , eight thousand twenty , eight thousand twenty one , eight thousand twenty two , eight thousand twenty three , eight thousand twenty four , eight thousand twenty five , eight thousand twenty six , eight thousand twenty seven , eight thousand twenty eight , eight thousand twenty nine , eight thousand thirty , eight thousand thirty one , eight thousand thirty two , eight thousand thirty three , eight thousand thirty four , eight thousand thirty five , eight thousand thirty six , eight 

## 3.3 เช็คข้อมูล

จำนวน Token (ความยาว) ของ ข้อมูลตัวอย่าง x ตัวแรก ใน Validation Set

In [35]:
len(data.valid_ds[0][0].data)

13017

Backpropagation Through Time (BPTT) คือจะเทรนไปทีละกี่ Token ถึงจะทำ Backpropagation หา Gradient และจำนวน Batch ใน Validation Set

In [36]:
data.bptt, len(data.valid_dl)

(70, 3)

กำหนด [Batch Size](https://www.bualabs.com/archives/729/what-is-batch-size-in-deep-neural-networks-how-to-adjust-machine-learning-model-accuracy-deep-learning-hyperparameter-tuning-ep-2/) = 64

In [0]:
bs=64

นำข้อมูลตัวอย่าง มาหารด้วยความยาว BPTT ว่าจะใส่ข้อความยาวเท่าไร และ Batch Size ว่าจะแบ่งเป็นกี่ข้อความ ที่จะ Feed ใส่พร้อม ๆ กัน

คำนวนได้ประมาณ 3 Mini-Batch ใน 1 Epoch

In [41]:
13017/70/bs

2.905580357142857

เราจะใช้ Iterator แบ่ง Validation Set ออกมาเป็น x, y 3 ชุดเลย จะได้เห็นชัด ๆ

In [0]:
it = iter(data.valid_dl)

x1, y1 = next(it)
x2, y2 = next(it)
x3, y3 = next(it)

it.close()

จะได้ ข้อมูล x ขนาด Batch Size x BPTT ออกมา และ Label y เท่ากับ x ตัวถัดไป สำหรับเทรน Language Model

In [69]:
x1.shape, y1.shape, x1[0, :10], y1[0, :10]

(torch.Size([64, 70]),
 torch.Size([64, 70]),
 tensor([ 2, 13,  9, 14, 10, 13,  9, 15, 10, 13]),
 tensor([13,  9, 14, 10, 13,  9, 15, 10, 13,  9]))

เช็คว่าครบไหม พบว่าเกิน

numel = Number of Elements

In [54]:
x1.numel() + x2.numel() + x3.numel()

13440

เพราะมีการใส่ Padding ให้ความยาวเท่ากันทุก Batch

In [55]:
x1.numel(), x2.numel(), x3.numel()

(4480, 4480, 4480)

## 3.4 ดูข้อมูล Explore Data

ดูรายการคำศัพท์ทั้งหมด ใน vocab Dictionary 

In [0]:
v = data.valid_ds.vocab

In [51]:
v.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'thousand',
 ',',
 'hundred',
 'nine',
 'eight',
 'one',
 'two',
 'three',
 'four',
 'five',
 'six',
 'seven',
 'twenty',
 'thirty',
 'forty',
 'fifty',
 'sixty',
 'seventy',
 'eighty',
 'ninety',
 'ten',
 'eleven',
 'twelve',
 'thirteen',
 'fourteen',
 'fifteen',
 'sixteen',
 'seventeen',
 'eighteen',
 'nineteen',
 'xxfake']

In [72]:
x1[0, :]

tensor([ 2, 13,  9, 14, 10, 13,  9, 15, 10, 13,  9, 16, 10, 13,  9, 17, 10, 13,
         9, 18, 10, 13,  9, 19, 10, 13,  9, 20, 10, 13,  9, 13, 10, 13,  9, 12,
        10, 13,  9, 29, 10, 13,  9, 30, 10, 13,  9, 31, 10, 13,  9, 32, 10, 13,
         9, 33, 10, 13,  9, 34, 10, 13,  9, 35, 10, 13,  9, 36, 10, 13])

In [73]:
y1[0, :]

tensor([13,  9, 14, 10, 13,  9, 15, 10, 13,  9, 16, 10, 13,  9, 17, 10, 13,  9,
        18, 10, 13,  9, 19, 10, 13,  9, 20, 10, 13,  9, 13, 10, 13,  9, 12, 10,
        13,  9, 29, 10, 13,  9, 30, 10, 13,  9, 31, 10, 13,  9, 32, 10, 13,  9,
        33, 10, 13,  9, 34, 10, 13,  9, 35, 10, 13,  9, 36, 10, 13,  9])

In [75]:
v.itos[13], v.itos[9], v.itos[14], v.itos[10], v.itos[13], v.itos[9], v.itos[15], v.itos[10] 

('eight', 'thousand', 'one', ',', 'eight', 'thousand', 'two', ',')

ดูเป็นข้อความ Text

In [71]:
v.textify(x1[0])

'xxbos eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight'

In [76]:
v.textify(x1[1])

', eight thousand forty six , eight thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine ,'

In [77]:
v.textify(y1[1])

'eight thousand forty six , eight thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine , eight'

In [81]:
v.textify(x2[0])

'thousand eighteen , eight thousand nineteen , eight thousand twenty , eight thousand twenty one , eight thousand twenty two , eight thousand twenty three , eight thousand twenty four , eight thousand twenty five , eight thousand twenty six , eight thousand twenty seven , eight thousand twenty eight , eight thousand twenty nine , eight thousand thirty , eight thousand thirty one , eight thousand thirty two ,'

In [82]:
v.textify(x3[0])

'eight thousand thirty three , eight thousand thirty four , eight thousand thirty five , eight thousand thirty six , eight thousand thirty seven , eight thousand thirty eight , eight thousand thirty nine , eight thousand forty , eight thousand forty one , eight thousand forty two , eight thousand forty three , eight thousand forty four , eight thousand forty five , eight thousand forty six , eight'

In [83]:
v.textify(x3[-1])

'ninety , nine thousand nine hundred ninety one , nine thousand nine hundred ninety two , nine thousand nine hundred ninety three , nine thousand nine hundred ninety four , nine thousand nine hundred ninety five , nine thousand nine hundred ninety six , nine thousand nine hundred ninety seven , nine thousand nine hundred ninety eight , nine thousand nine hundred ninety nine xxbos eight thousand one , eight'

In [85]:
data.show_batch(ds_type=DatasetType.Valid)

idx,text
0,"thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine , eight thousand sixty , eight thousand sixty"
1,"eight , eight thousand eighty nine , eight thousand ninety , eight thousand ninety one , eight thousand ninety two , eight thousand ninety three , eight thousand ninety four , eight thousand ninety five , eight thousand ninety six , eight thousand ninety seven , eight thousand ninety eight , eight thousand ninety nine , eight thousand one hundred , eight thousand one hundred one , eight thousand one"
2,"thousand one hundred twenty four , eight thousand one hundred twenty five , eight thousand one hundred twenty six , eight thousand one hundred twenty seven , eight thousand one hundred twenty eight , eight thousand one hundred twenty nine , eight thousand one hundred thirty , eight thousand one hundred thirty one , eight thousand one hundred thirty two , eight thousand one hundred thirty three , eight thousand"
3,"three , eight thousand one hundred fifty four , eight thousand one hundred fifty five , eight thousand one hundred fifty six , eight thousand one hundred fifty seven , eight thousand one hundred fifty eight , eight thousand one hundred fifty nine , eight thousand one hundred sixty , eight thousand one hundred sixty one , eight thousand one hundred sixty two , eight thousand one hundred sixty three"
4,"thousand one hundred eighty three , eight thousand one hundred eighty four , eight thousand one hundred eighty five , eight thousand one hundred eighty six , eight thousand one hundred eighty seven , eight thousand one hundred eighty eight , eight thousand one hundred eighty nine , eight thousand one hundred ninety , eight thousand one hundred ninety one , eight thousand one hundred ninety two , eight thousand"


# 4. Model

## 4.1 Fully Connected Model 1 Layer

In [87]:
data = labeled_splitted.databunch(bs=bs, bptt=3)
data

TextLMDataBunch;

Train: LabelList (1 items)
x: LMTextList
xxbos eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight thousand eighteen , eight thousand nineteen , eight thousand twenty , eight thousand twenty one , eight thousand twenty two , eight thousand twenty three , eight thousand twenty four , eight thousand twenty five , eight thousand twenty six , eight thousand twenty seven , eight thousand twenty eight , eight thousand twenty nine , eight thousand thirty , eight thousand thirty one , eight thousand thirty two , eight thousand thirty three , eight thousand thirty four , eight thousand thirty five , eight thousand thirty six , eight 

In [91]:
x, y = data.one_batch()
x.shape, y.shape

(torch.Size([64, 3]), torch.Size([64, 3]))

In [92]:
x[:10, :], y[:10, :]

(tensor([[10, 13,  9],
         [ 9, 23, 13],
         [12, 10, 13],
         [13,  9, 14],
         [11, 24, 17],
         [10, 13,  9],
         [11, 34, 10],
         [23, 18, 10],
         [ 9, 15, 11],
         [16, 11, 18]]), tensor([[13,  9, 17],
         [23, 13, 10],
         [10, 13,  9],
         [ 9, 14, 11],
         [24, 17, 10],
         [13,  9, 14],
         [34, 10, 13],
         [18, 10, 13],
         [15, 11, 26],
         [11, 18, 10]]))